In [29]:
import tensorflow as tf
import numpy as np

height = 28
width = 28
channel = 1

(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = tf.keras.datasets.mnist.load_data()
X_train = X_train_raw.astype(np.float32).reshape(-1, height * width)
X_test = X_test_raw.astype(np.float32).reshape(-1, height * width)
y_train = y_train_raw.astype(np.int32)
y_test = y_test_raw.astype(np.int32)

In [30]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, height * width), name="X")
X_reshaped = tf.reshape(X, shape=(-1, width, height, channel))
y = tf.placeholder(tf.int32, shape=[None], name="y")
training = tf.placeholder_with_default(False, shape=[], name="training")

# Neural Network
conv1 = tf.layers.conv2d(X_reshaped, filters=32, kernel_size=3, strides=1, padding="SAME")
conv2 = tf.layers.conv2d(conv1, filters=64, kernel_size=3, strides=1, padding="SAME")
pool3 = tf.layers.max_pooling2d(conv2, pool_size=[2,2], strides=2, padding="SAME")
pool3_flat = tf.reshape(pool3, shape=[-1, 64 * 14 * 14])
pool3_flat_drop = tf.layers.dropout(pool3_flat, rate=0.5, training=training)
hidden1 = tf.layers.dense(pool3_flat_drop, 128, activation=tf.nn.relu)
hidden1_drop = tf.layers.dropout(hidden1, rate=0.5, training=training)
logits = tf.layers.dense(hidden1_drop, 10, name="logits")

# Loss
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y, name="xentropy")
loss = tf.reduce_mean(xentropy, name="loss")
loss_summary = tf.summary.scalar("loss_summary", loss)

# Train
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")

# Evaluation
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
accuracy_summary = tf.summary.scalar("accuracy_summary", accuracy)

init = tf.global_variables_initializer()
save = tf.train.Saver()

In [31]:
batch_size = 100
n_epoch = 10

X_train = X_train[:1000]

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epoch):
        for batch_index in range(len(X_train) // batch_size):
            rdm_idx = np.random.permutation(len(X_train))
            X_batch, y_batch = X_train[rdm_idx[:batch_size]], y_train[rdm_idx[:batch_size]]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
        accuracy_train = sess.run(accuracy, feed_dict={X: X_train, y: y_train})
        accuracy_test = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
        print("Training Accuracy: {:.2f}%\tTest Accuracy: {:.2f}%".format(accuracy_train * 100, accuracy_test * 100))

InvalidArgumentError: First dimension of predictions 1000 must match length of targets 60000
	 [[Node: in_top_k/InTopKV2 = InTopKV2[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](logits/BiasAdd, _arg_y_0_1, in_top_k/InTopKV2/k)]]

Caused by op 'in_top_k/InTopKV2', defined at:
  File "/Users/howard/anaconda3/envs/mlbook/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/howard/anaconda3/envs/mlbook/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/Users/h